# Исследование надежности заемщиков.



**Описание проекта** <a id='intro'></a>

**Заказчик:** кредитный отдел банка. 

**Задача:**
Необходимо разобраться, как влияют различные факторы - характеристики клиента на факт погашения кредита в срок.

**Вопросы, которые обозначил заказчик перед аналитиками:**
* Есть ли зависимость между количеством детей и возвратом кредита в срок?
* Есть ли зависимость между семейным положением и возвратом кредита в срок?
* Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
* Как разные цели кредита влияют на его возврат в срок?

**Вводные:**
Входные данные от банка - статистика о платёжеспособности клиентов в таблице формата `.csv`.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

Кредитный скоринг (от англ. score «оценка»; в т.ч. гл.) — система оценки кредитоспособности (кредитных рисков) потенциального клиента, основанная на численных статистических методах. 

**Скоринг** заключается в присвоении баллов по итогам заполнения анкеты заемщика, разработанной оценщиками кредитных рисков (андеррайтерами). По результатам набранных баллов системой автоматически принимается решение об одобрении или отказе в выдаче кредита.

**Описание данных**

1. children — количество детей в семье
2. days_employed — общий трудовой стаж в днях
3. dob_years — возраст клиента в годах
4. education — уровень образования клиента
5. education_id — идентификатор уровня образования
6. family_status — семейное положение
7. family_status_id — идентификатор семейного положения
8. gender — пол клиента
9. income_type — тип занятости
10. debt — имел ли задолженность по возврату кредитов
11. total_income — ежемесячный доход
12. purpose — цель получения кредита

[К заключению по проекту](#conclusion)

### Шаг 1. Обзор данных

Необходимо составить первое представление о полученных данных от кредитного отдела банка. 

* Для работы с табличными файлами импортируем библиотеку `pandas`. 
* Для игнорирования предупреждений импортируем библиотеку `warnings`.


In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

Прочитаем файл `data (1).csv` из папки `/datasets/data.csv` и сохраним его в переменной `df`. Изучем наполнение файла, типы данных, количество и т.д.


In [2]:
df = pd.read_csv('/datasets/data.csv')
display(df.sample(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
14920,1,NaN,28,высшее,0,Не женат / не замужем,4,F,компаньон,0,NaN,покупка своего жилья
18327,1,-4109.464697,29,среднее,1,женат / замужем,0,M,сотрудник,0,292124.203385,приобретение автомобиля
19336,2,-3230.724011,45,среднее,1,женат / замужем,0,M,сотрудник,0,142224.177570,на покупку своего автомобиля
8703,1,-1600.798781,22,среднее,1,женат / замужем,0,M,компаньон,0,52832.041117,строительство недвижимости
13714,0,-598.430007,23,неоконченное высшее,2,женат / замужем,0,M,госслужащий,0,203711.266197,получение дополнительного образования
17252,0,377113.431713,53,высшее,0,женат / замужем,0,F,пенсионер,0,170433.137385,покупка жилья для семьи
6297,0,-3090.011801,32,среднее,1,женат / замужем,0,F,сотрудник,0,115334.548630,покупка недвижимости
12292,0,386259.675109,64,среднее,1,женат / замужем,0,M,пенсионер,0,55758.042205,операции со своей недвижимостью
12799,0,-7345.453207,44,среднее,1,женат / замужем,0,F,сотрудник,0,166521.090180,операции со своей недвижимостью
5786,0,-10477.931668,54,среднее,1,женат / замужем,0,F,госслужащий,0,105878.505972,приобретение автомобиля


Проверив случайной выборкой, сразу видим ошибки в столбце `days_employed` - стаж в вещественном формате и есть отрицательные значения. Нужно привести к целым абсолютам. В столбце `education` записи в разных регистрах. 


In [3]:
df.info() #проверим общую информацию о содержании датасета

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
df.describe() #проверим статистические данные датафрейма.

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [5]:
df.columns #проверим написание колонок

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

Итак, проведя первичный обзор, видим следующее.

В столбце 12 колонок с данными
1. `children` — количество детей в семье
2. `days_employed` — общий трудовой стаж в днях
3. `dob_years` — возраст клиента в годах
4. `education` — уровень образования клиента
5. `education_id` — идентификатор уровня образования
6. `family_status` — семейное положение
7. `family_status_id` — идентификатор семейного положения
8. `gender` — пол клиента
9. `income_type` — тип занятости
10. `debt` — имел ли задолженность по возврату кредитов
11. `total_income` — ежемесячный доход
12. `purpose` — цель получения кредита

Названия колонок написаны в змеином стиле в нижнем регистре. Исправления названий не требуется. 

Количество значений в столбцах различается. Значит, в данных есть пропущенные значения в колонках `days_employed` и `total_income`.

### Шаг 2.1 Заполнение пропусков

In [6]:
display(df.isna().sum()) # посчитаем пропуски

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [7]:
days_employed_ratio = df['days_employed'].isna().sum()/df['children'].count()

print (f'Доля пропусков в столбцах с трудовым стажем и ежемесячным доходом: {days_employed_ratio:.1%}')


Доля пропусков в столбцах с трудовым стажем и ежемесячным доходом: 10.1%


Действительно есть пропущенные значения в двух столбцах `days_employed` и `total_income`. 
Доля пропусков в обоих столбцах одинаковая и составляет около `10%` от всех данных. Строки удалять нельзя, т.к. это существенная часть выборки, которую необходимо использовать в расчетах.

Предполагаю, что не все люди пожелали вписать данные о стаже и заработке в анкету. Возможно, в связи с неофициальным трудоустройством и отсутствием подтверждающих документов. Либо это могли быть несовершеннолетние, которые еще не работали ранее. 

Эти столбцы содержат количественные переменные (вещественный тип). Заполним пропуски `медианным значением`. Нельзя брать среднее значение для заполнения пропусков, т.к. это сильно повлияет на репрезентативность выбороки из-за возможных максимальных отклонений текущих данных. Трудовой стаж в днях может сильно отличаться - например, у людей пенисонного возраста он сильно выше чем у людей среднего возраста или молодежи, которые обычно берут кредиты. 75 %-тиль показывает, что у нас 25% кредиторов старше 53 лет.
Или, например, в колонке с зарплатой, могут быть зарплаты крупных руководителей.


In [8]:
days_employed_median = df['days_employed'].median() # найдем медиану в столбце с трудовым стажем
total_income_median = df['total_income'].median() # найдем медиану в столбце с ежемесячным доходом
print (days_employed_median)
print (total_income_median)

-1203.369528770489
145017.93753253992


In [9]:
total_income_negative = df[df['total_income']<0]['total_income'].count() #проверим количество отрицательных значений
print (total_income_negative)

0


In [10]:
days_employed_negative = df[df['days_employed']<0]['days_employed'].count() #проверим количество отрицательных значений
days_employed_negative

15906

Видим, что медианный трудовой стаж вышел отрицательный. Таких значений очень много, более 50% от выборки. Возможно, на это повлиял человеческий фактор или был технический баг. Нужно скорректировать эту аномалию. Приведем все данные к абсолютному значению и проверим количество отрицательных значений. 

### Шаг 2.2 Проверка данных на аномалии и исправления.

Переведем строки с отрицательными значениями в абсолютные величины.

In [11]:
df['days_employed'] = abs(df['days_employed']) #переведем в абсолютные значения
days_employed_negative = df[df['days_employed']<0]['days_employed'].count()
days_employed_negative #проверим, остались ли отрицательные числа

0

In [12]:
children_negative = df[df['children']<0]['children'].count() #проверим наличие минусов в количестве детей
children_negative

47

Вывод, в столбце `children` 47 строк с отрицательными значениеми. Проверка показала, что практически все эти люди живут в браке. Предположим что повлиял человеский фактор, и у них всех есть дети, поэтому заменим этот - на +

In [13]:
df['children'] = abs(df['children']) #переведем в абсолютные значения
children_negative = df[df['children']<0]['children'].count()
children_negative #проверим, остались ли отрицательные числа

0

In [14]:
print(df['children'].max())  #уточним максимальное значение детей, возможно там есть ошибка
df = df.sort_values(by='children')
print (df['children'].tail(10)) #проверим много ли значений 20 в этом столбце
# обнаружено эсктримальное значение - предположим что это ошибка с доп. нулем и проведем замену на 2

20
18493    20
10194    20
18476    20
20038    20
17509    20
3302     20
5020     20
15313    20
16795    20
15812    20
Name: children, dtype: int64


In [15]:
df['children']=df['children'].replace(20, 2)  # предположим, что была ошибка, добавлен 0, заменим на 2
df = df.sort_values(by='children')
print(df['children'].tail(10))

5991     4
21156    5
20452    5
20837    5
16211    5
3979     5
4397     5
7866     5
15916    5
15822    5
Name: children, dtype: int64


Проверка столбца с детьми на максимальное значение выдала результат - 20. Скорее всего при заполнении анкеты оператор допустил ошибку и прибавил к 2ке - "0". После замены в столбце `children` аномальных значений с 20 детьми, фиксируем максимальное значение 5 детей только в 9 случаях. Это норма.

Заменим пропуски в столбцах с стажем и доходом. Эти данные нам еще пригодятся для итогового вывода по задаче.

In [16]:
df['days_employed'] = df['days_employed'].fillna(value = days_employed_median) #заменим пропуски
df['total_income'] = df['total_income'].fillna(value = total_income_median)
print(df.isna().sum()) # посчитаем пропуски

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64


Пропусков и отрицательных значений не осталось!

### Шаг 2.3. Изменение типов данных.

Переведем все ячейки трудового стажа и доходы клиента в целочисленные значения integer

In [17]:
df['total_income'] = df['total_income'].astype('int')
df['days_employed'] = df['days_employed'].astype('int')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21525 entries, 9612 to 15822
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int64 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB


Как мы заметили ранее, столбцы `education` и `family_status` заполнены разными регистрами. Приведем все строки в нижний регистр.

In [18]:
df['education'] = df['education'].str.lower()
df['family_status'] = df['family_status'].str.lower()
#проверим строки на регистр 
print(df['education'].unique())
print(df['family_status'].unique())

['среднее' 'высшее' 'начальное' 'неоконченное высшее' 'ученая степень']
['женат / замужем' 'вдовец / вдова' 'в разводе' 'гражданский брак'
 'не женат / не замужем']


### Шаг 2.4. Удаление дубликатов.

Найдем явные дубликаты и избавимся от них.

In [19]:
print(df.duplicated().sum()) # подсчёт явных дубликатов

71


In [20]:
df=df.drop_duplicates().reset_index(drop=True) # удаление явных дубликатов (с удалением старых индексов и формированием новых)
print(df.duplicated().sum()) # проверка на отсутствие дубликатов

0


Теперь поищем неявные дубликаты в столбце `family_status`

In [21]:
df = df.sort_values(by='family_status')
# print (df['family_status'].head (20)) проверка сортировки
print(df['family_status'].unique()) # Просмотр уникальных названий семейного положения

['в разводе' 'вдовец / вдова' 'гражданский брак' 'женат / замужем'
 'не женат / не замужем']


In [22]:
df = df.sort_values(by='gender')
print(df['gender'].unique()) # Просмотр уникальных названий пола

['F' 'M' 'XNA']


In [23]:
print(df[df['gender'] == 'XNA']['gender'].count())

1


In [24]:
print(df['debt'].unique()) # Просмотр уникальных названий статуса задолженности

[0 1]


Проверили некоторые столбцы датасета на не явные дубликаты и ошибки. Значительные ошибки, которые могут повлтиять на результат исследования пока не обнаружили.

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Создадим два новых датафрейма **словаря** со столбцами:
`education_id` и `education` — в первом;
`family_status_id` и `family_status` — во втором.
        
		


In [25]:
education_dict = df[['education_id', 'education']]
family_status_dict = df[['family_status_id', 'family_status']]
display(education_dict.head())
display(family_status_dict.head())

,education_id,education
16972,1,среднее
3869,1,среднее
3871,1,среднее
3872,1,среднее
3873,1,среднее


,family_status_id,family_status
16972,3,в разводе
3869,0,женат / замужем
3871,0,женат / замужем
3872,0,женат / замужем
3873,0,женат / замужем


В «словарях» мы заметили большое количество не явных дубликатов. Их нужно будет удалить. Применим знакомую цепочку методов: `drop_duplicates()` и `reset_index()`.


In [26]:
education_dict=education_dict.drop_duplicates().reset_index(drop=True) # удаление явных дубликатов (с удалением старых индексов и формированием новых)
family_status_dict=family_status_dict.drop_duplicates().reset_index(drop=True) 
display(education_dict)
display(family_status_dict)

,education_id,education
0,1,среднее
1,2,неоконченное высшее
2,0,высшее
3,3,начальное
4,4,ученая степень


,family_status_id,family_status
0,3,в разводе
1,0,женат / замужем
2,4,не женат / не замужем
3,1,гражданский брак
4,2,вдовец / вдова


После удаления дубликатов представление данных с оценками id стало компактнее. Таблицу будет легче группировать по id.

Удалим из исходного датафрейма столбцы `education` и `family_status`, оставив только их идентификаторы: `education_id` и `family_status_id`. 


In [27]:
df = df.drop(['education', 'family_status'], axis = 1)
df.columns # проверим какие остались столбцы

Index(['children', 'days_employed', 'dob_years', 'education_id',
       'family_status_id', 'gender', 'income_type', 'debt', 'total_income',
       'purpose'],
      dtype='object')

### Шаг 2.6. Категоризация дохода.

Категоризируем - столбец ежемесячный доход. Создадим столбец с группами `total_income_category`. Такую группу легче будет анализировать чем разрозненные данные.


In [28]:
def total_income_category(income):
    if income <=30000:
        i = 'F'
    elif income >30000 and income<=50000:
        i = 'E'
    elif income >50000 and income<=100000:
        i = 'D'
    elif income >100000 and income<=200000:
        i = 'C'        
    elif income >200000 and income<=1000000:
        i = 'B'
    else:
        i = 'A'
    return i

df['total_income_category']= df['total_income'].apply(total_income_category)
display(df.head())

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
16972,1,5907,47,1,3,F,госслужащий,0,314656,операции с недвижимостью,B
3869,0,507,46,1,0,F,сотрудник,0,73986,строительство жилой недвижимости,D
3871,0,210,62,1,0,F,сотрудник,0,98635,образование,D
3872,0,369236,52,1,0,F,пенсионер,0,48975,покупка жилой недвижимости,E
3873,0,276,43,1,0,F,сотрудник,0,104862,получение высшего образования,C


### Шаг 2.7. Категоризация целей кредита.

Категоризируем цели кредита. Создадим 4 основных категории и 5-ю "Другое" для проверки. Запишем их в новый столбец `purpose_category`

In [29]:
df = df.sort_values(by='purpose')
print(df['purpose'].unique()) # Просмотр уникальных названий цели кредита

['автомобили' 'автомобиль' 'высшее образование'
 'дополнительное образование' 'жилье' 'заняться высшим образованием'
 'заняться образованием' 'на покупку автомобиля'
 'на покупку подержанного автомобиля' 'на покупку своего автомобиля'
 'на проведение свадьбы' 'недвижимость' 'образование' 'операции с жильем'
 'операции с коммерческой недвижимостью' 'операции с недвижимостью'
 'операции со своей недвижимостью' 'покупка жилой недвижимости'
 'покупка жилья' 'покупка жилья для сдачи' 'покупка жилья для семьи'
 'покупка коммерческой недвижимости' 'покупка недвижимости'
 'покупка своего жилья' 'получение высшего образования'
 'получение дополнительного образования' 'получение образования'
 'приобретение автомобиля' 'профильное образование' 'ремонт жилью'
 'свадьба' 'свой автомобиль' 'сделка с автомобилем'
 'сделка с подержанным автомобилем' 'строительство жилой недвижимости'
 'строительство недвижимости' 'строительство собственной недвижимости'
 'сыграть свадьбу']


In [30]:
def purpose_category(purpose):
    if purpose.__contains__('автом'):
        p = 'операции с автомобилем'
    elif purpose.__contains__('недвиж'):
        p = 'операции с недвижимостью'
    elif purpose.__contains__('свадьб'):
        p = 'проведение свадьбы' 
    elif purpose.__contains__('образов'):
        p = 'получение образования'
    else:
        p = 'Другое'
    return p

df['purpose_category']=df['purpose'].apply(purpose_category)
display(df.head())

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
2367,0,339396,64,0,0,F,пенсионер,0,132186,автомобили,C,операции с автомобилем
379,0,458,21,1,4,F,сотрудник,0,89727,автомобили,D,операции с автомобилем
824,0,3867,53,1,0,F,сотрудник,0,211179,автомобили,B,операции с автомобилем
2363,0,401291,57,1,0,F,пенсионер,0,276693,автомобили,B,операции с автомобилем
1192,0,574,40,1,0,F,сотрудник,0,161158,автомобили,C,операции с автомобилем


Проверим какие категории получились. Возможно у нас прописалась категория "Другое"

In [31]:
print(df['purpose_category'].unique()) # Просмотр уникальных названий категории цели кредита

['операции с автомобилем' 'получение образования' 'Другое'
 'проведение свадьбы' 'операции с недвижимостью']


В категорию `Другое` прописалась цель, которую мы не заметили при создании функции. Проверим эту цель.

In [32]:
df_1 = df[df['purpose_category'] == 'Другое']
display(df_1[['purpose_category', 'purpose']].head(5))

,purpose_category,purpose
4243,Другое,жилье
8335,Другое,жилье
5414,Другое,жилье
5408,Другое,жилье
18433,Другое,жилье


Перезапишем код, чтобы удалить данные из категории "Другое"

In [33]:
def purpose_category(purpose):
    if purpose.__contains__('автом'):
        p = 'операции с автомобилем'
    elif purpose.__contains__('недвиж') or purpose.__contains__('жил'): # добавим проверку на содержание 'жил'
        p = 'операции с недвижимостью'
    elif purpose.__contains__('свадьб'):
        p = 'проведение свадьбы' 
    elif purpose.__contains__('образов'):
        p = 'получение образования'
    else:
        p = 'Другое'
    return p
df['purpose_category']=df['purpose'].apply(purpose_category)
print(df['purpose_category'].unique()) # Просмотр уникальных названий категории цели кредита
print ('_______________________________________________________________________________')
print (df.columns) #проверим новые столбцы

['операции с автомобилем' 'получение образования'
 'операции с недвижимостью' 'проведение свадьбы']
_______________________________________________________________________________
Index(['children', 'days_employed', 'dob_years', 'education_id',
       'family_status_id', 'gender', 'income_type', 'debt', 'total_income',
       'purpose', 'total_income_category', 'purpose_category'],
      dtype='object')


Теперь все как надо - выделили 4 категории в целях кредита.

К датасету прибавиись новые колонки с категориями: `total_income_category` и `purpose_category`.

**Выводы**

Предобработка обнаружила проблемы в данных:

- пропуски в данных - заполнили медианным значением
- отрицательные значения - привели к абсолютным
- аномально высокие значения - исправили
- столбцы с трудовым стажем и доходом имели вещественный тип данных - поменяли на целочисленный
- удалили явные дубликаты (71 строка)
- выявили и исправили не явные дубликаты

Дополнительно создали 2 словаря с групипровкой по ID для семейного положения и уровня образования
Провели категоризацию столбцов с ежемесячным доходом и целями на кредит. 

Теперь можно перейти к ответам на вопросы заказчика.

### Ответы на вопросы.

#### Вопрос 1:
	Необходимо проверить зависимость между количеством детей и возвратом кредита в срок.


Создадим сводную таблицу с группировкой по количеству детей. 
В значения поместим данные по количеству клиентов с задолженностью, где
```
0 - не было задолженности по кредиту
1 - имелась задолженность по кредиту
```

In [34]:
df_pivot = df.pivot_table(index=['children'], columns='debt', values='family_status_id', aggfunc='count')
df_pivot

debt,0,1
children,,
0,13028.0,1063.0
1,4410.0,445.0
2,1926.0,202.0
3,303.0,27.0
4,37.0,4.0
5,9.0,NaN


По данным из сводной таблицы создадим новый датафрейм для удобства проведения расчетов. 

In [35]:
debt_ = {'children':[0, 1, 2, 3, 4, 5],
         'dept_0':[13028, 4410, 1926, 303, 37, 9],
         'dept_1':[1063, 445, 202, 27, 4, 0]
        }
df_debt = pd.DataFrame(debt_)
df_debt

,children,dept_0,dept_1
0,0,13028,1063
1,1,4410,445
2,2,1926,202
3,3,303,27
4,4,37,4
5,5,9,0


Расчитаем долю клиентов не возвративших кредит в срок от общего количества клиентов. 
Используем новую функцию `debt_ratio`, которую, далее, будем использовать для анализа влияния других факторов.

In [36]:
def debt_ratio(row):
    d = (row['dept_1']/(row['dept_0']+row['dept_1'])*100).round(1)
    return d

df_debt['debt_ratio,%']=df_debt.apply(debt_ratio, axis=1) # запишем новый столбец с долей неплательщиков
df_debt

,children,dept_0,dept_1,"debt_ratio,%"
0,0,13028,1063,7.5
1,1,4410,445,9.2
2,2,1926,202,9.5
3,3,303,27,8.2
4,4,37,4,9.8
5,5,9,0,0.0


**Вывод**

Если сравнить клиентов с разным количеством детей (от 1го до 5) и клиентов без детей, можно сделать следующие выводы:
1. Клиенты без детей имеют высокую платежеспособность. Доля клиентов с задолженностью составляет всего 7,5%.
2. Наибольшая доля невозврата кредитов у клиентов имеющих одного, двух или четырех детей. Она составляет более 9%. 
3. Результат по клиентам с 5-ю детьми можно не учитывать в итоговом выводе. Их количество в выборке ничтожно мало (менее 0,1%).

Таким образом, можно сделать вывод, что клиенты без детей чаще возвращают кредиты, чем клиенты с детьми. 

Есть гипотезы, которые нужно дополнительнро проверять, а также учитывать в модели скорринга другие факторы (возраст, доход, семейное положение и т.д.).

Гипотезы:
Вероятно среди клиентов с детьми есть родители одиночки, которые имеют низкий доход на семью. Либо, как мы видели в начале анализа, более 25% выборки - это люди старше 53 лет. Часть из них не работающие пенсионеры. Скорее всего у большинства из них взрослые дети которые совместно с ними не проживают. 
Считаю, что при учете этого фактора для модели скорринга нужно дополнительно проверять семейное положение и другие факторы. 

#### Вопрос 2:

Необходимо оценить зависимость между семейным положением и возвратом кредита в срок.


In [37]:
df_pivot = df.pivot_table(index=['family_status_id'], columns='debt', values='education_id', aggfunc='count')
df_pivot

debt,0,1
family_status_id,,
0,11408,931
1,3763,388
2,896,63
3,1110,85
4,2536,274


In [38]:
debt_ = {'family_status_id':[0, 1, 2, 3, 4],
         'dept_0':[11408, 3763, 896, 1110, 2536],
         'dept_1':[931, 388, 63, 85, 274]
        }
df_debt = pd.DataFrame(debt_)
df_debt

,family_status_id,dept_0,dept_1
0,0,11408,931
1,1,3763,388
2,2,896,63
3,3,1110,85
4,4,2536,274


In [39]:
def debt_ratio(row):
    d = (row['dept_1']/(row['dept_0']+row['dept_1'])*100).round(1)
    return d

df_debt['debt_ratio,%']=df_debt.apply(debt_ratio, axis=1)
df_debt

,family_status_id,dept_0,dept_1,"debt_ratio,%"
0,0,11408,931,7.5
1,1,3763,388,9.3
2,2,896,63,6.6
3,3,1110,85,7.1
4,4,2536,274,9.8


In [40]:
df_family = family_status_dict.merge(df_debt,on='family_status_id',how='left')
df_family

,family_status_id,family_status,dept_0,dept_1,"debt_ratio,%"
0,3,в разводе,1110,85,7.1
1,0,женат / замужем,11408,931,7.5
2,4,не женат / не замужем,2536,274,9.8
3,1,гражданский брак,3763,388,9.3
4,2,вдовец / вдова,896,63,6.6


**Вывод**

Если сравнить клиентов с разным семейным статусом, можно сделать следующие выводы о их платежеспособности: 

1. Группа с не женатыми/не замужними клиентами имеют максимальный % неплатежеспособности.  9,8% клиентов имеют задолженность перед банком. Высокий % должников наблюдается также в группе "гражданский брак" (9,3%).
2. Должники в группах "женат/замужем" и "в разводе" составляют около 7%.
3. Самая низкая доля должников в группе "вдовец/вдова" - 6,6%.

Таким образом, клиентам не состоявшим ранее в официальном браке, стоит уделить повышенное внимание при кредитном скорринге. Риск по таким клиентам максимальный.

Для более достоверного и полного анализа платежеспособности клиентов с разным семейным статусом, стоит проанализировать их в разрезе других групп (типе занятости, ежемесячном доходе, общем трудовом стаже и т.д.)

#### Вопрос 3:

Необходимо проанализировать, есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [41]:
df_pivot = df.pivot_table(index=['total_income_category'], columns='debt', values='education_id', aggfunc='count')
df_pivot

debt,0,1
total_income_category,,
A,23,2
B,4685,356
C,10896,1029
D,3760,331
E,329,21
F,20,2


In [42]:
debt_ = {'total_income_category':['A', 'B', 'C', 'D', 'E', 'F'],
         'dept_0':[23, 4685, 10896, 3760, 329, 20],
         'dept_1':[2, 356, 1029, 331, 21, 2]
        }
df_debt = pd.DataFrame(debt_)
df_debt

,total_income_category,dept_0,dept_1
0,A,23,2
1,B,4685,356
2,C,10896,1029
3,D,3760,331
4,E,329,21
5,F,20,2


In [43]:
def debt_ratio(row):
    d = round(row['dept_1']/(row['dept_0']+row['dept_1'])*100, 1)
    return d

df_debt['debt_ratio,%'] = df_debt.apply(debt_ratio, axis=1)
df_debt

,total_income_category,dept_0,dept_1,"debt_ratio,%"
0,A,23,2,8.0
1,B,4685,356,7.1
2,C,10896,1029,8.6
3,D,3760,331,8.1
4,E,329,21,6.0
5,F,20,2,9.1


In [44]:
income_category = {'total_income_category':['A', 'B', 'C', 'D', 'E', 'F'],
         'income_level':['1000001 и выше', '200001–1000000', '100001–200000', '50001–100000', '30001–50000', '0–30000'],
         }
df_income_category = pd.DataFrame(income_category)
df_income_category

,total_income_category,income_level
0,A,1000001 и выше
1,B,200001–1000000
2,C,100001–200000
3,D,50001–100000
4,E,30001–50000
5,F,0–30000


In [45]:
df_income_level = df_income_category.merge(df_debt,on='total_income_category',how='left')
df_income_level

,total_income_category,income_level,dept_0,dept_1,"debt_ratio,%"
0,A,1000001 и выше,23,2,8.0
1,B,200001–1000000,4685,356,7.1
2,C,100001–200000,10896,1029,8.6
3,D,50001–100000,3760,331,8.1
4,E,30001–50000,329,21,6.0
5,F,0–30000,20,2,9.1


**Вывод**

Если сравнить клиентов с уровнем дохода, можно сделать следующие выводы о их платежеспособности: 

1. В группе клиентов с уровнем дохода ниже 30 тыс.рублей находятся более 9% должников. Это макисмальная доля среди групп.
2. Высокий % должников наблюдается также в группе с доходом от 100 тыс. до 200 тыс.рублей. Доля имеющих задолжненность составляет более 8,6%. 
3. Самая низкая доля должников в группе с доходом от 30 тыс до 50 тыс.рублей. Она равна 6%.
4. Клиентов получающих доход более милиона рублей в месяц ничтожно мало для выборки. Поэтому эта группа не репрезентативна для анализа. 

Таким образом, клиентам имеющим ежемесячный доход до 30 тыс.руб., стоит уделить повышенное внимание при кредитном скорринге, также как и клиентам с доходом от 100 до 200 тыс. рублей. Риск по таким клиентам максимальный.

Для более достоверного и полного анализа платежеспособности клиентов с разным уровнем дохода, следует также опираться на другие факторы (тип занятости, семейный статус, общий трудовой стаж и т.д.)


#### Вопрос 4:

Вясним как разные цели кредита влияют на его возврат в срок.

In [46]:
df_pivot = df.pivot_table(index=['purpose_category'], columns='debt', values='education_id', aggfunc='count')
df_pivot

debt,0,1
purpose_category,,
операции с автомобилем,3903,403
операции с недвижимостью,10029,782
получение образования,3643,370
проведение свадьбы,2138,186


In [47]:
debt_ = {'purpose_category':['операции с автомобилем', 'операции с недвижимостью', 'получение образования', 'проведение свадьбы'],
         'dept_0':[3903, 10029, 3643, 2138],
         'dept_1':[403, 782, 370, 186]
        }
df_debt = pd.DataFrame(debt_)
df_debt

,purpose_category,dept_0,dept_1
0,операции с автомобилем,3903,403
1,операции с недвижимостью,10029,782
2,получение образования,3643,370
3,проведение свадьбы,2138,186


In [48]:
def debt_ratio(row):
    d = round(row['dept_1']/(row['dept_0']+row['dept_1'])*100, 1)
    return d
df_debt['debt_ratio,%']=df_debt.apply(debt_ratio, axis=1)
df_debt

,purpose_category,dept_0,dept_1,"debt_ratio,%"
0,операции с автомобилем,3903,403,9.4
1,операции с недвижимостью,10029,782,7.2
2,получение образования,3643,370,9.2
3,проведение свадьбы,2138,186,8.0


**Вывод**

Проведя анализ клиентов с разными целями кридита и их корреляции с возвратом кредита, можно сделать следующие выводы:

1. Наибольшее количество должников в группе клиентов с целями кредитования на операции с автомобилем и получение образования. Доля таких клиентов более 9%.
2. Наибольший % клиентов с возвратом кредитных средств в группе "операции с недвижимостью". Около 93% клиентов не имеют задолженности перед банком. Вероятно, что на возвращаемость кредита влияет страхование ипотеки и обременение недвижимости банком.
 
Необходимо учитывать цели кредита при кредитном скорринге. При этом, рассматривая рисковые группы клиентов, указанные выше, уделять повышенное внимание и к другим факторам (тип занятости, семейный статус, общий трудовой стаж и т.д.)

## Общий вывод:

<a id='conclusion'></a>
**Оценив полученные от заказчика [данные](#intro) по кредитной истории и характеристикам клиентов, а также проведя анализ влияния различных факторов на возвратность кредитов было установлено:**
1. Если сравнить клиентов с разным количеством детей (от 1го до 5) и клиентов без детей, можно сделать следующие выводы:
- Клиенты без детей имеют высокую платежеспособность. Кредитный рейтинг по такой группе можно повысить. 
- И наоборот, рисковыми являются клиенты имеющие одного, двух или четырех детей. Кредитный рейтинг по такой группе необходимо понизить.

2. Если сравнить клиентов с разным семейным статусом, можно сделать следующие выводы о их платежеспособности:
- Клиенты, имеющие семейный статус "женат/замужем", "в разводе" или "вдовец/вдова" имеют высокую платежеспособность. Кредитный рейтинг по такой группе можно повысить. 
- Клиентам не состоявшим ранее в официальном браке (статусы "не женат/не замужем", "гражданский брак") стоит уделить повышенное внимание при кредитном скорринге. Риск по таким клиентам максимальный.

3. Если сравнить клиентов с уровнем дохода, можно сделать следующие выводы о их платежеспособности:
- Самая низкая доля должников среди клиентов с доходом от 30 тыс до 50 тыс.рублей. Кредитный рейтинг по такой группе можно повысить. 
- Клиентам, имеющим ежемесячный доход до 30 тыс.руб., стоит уделить повышенное внимание при кредитном скорринге, также как и клиентам с доходом от 100 до 200 тыс. рублей. Риск по таким клиентам максимальный. Кредитный рейтинг необходимо понизить.

4. Проведя анализ платежеспособности клиентов с разными целями кридитования, можно сделать следующие выводы:
- Наибольшая доля клиентов, возвращающих кредитные средствв, берут кредиты с целью "операции с недвижимостью". Около 93% клиентов не имеют задолженности перед банком. Вероятно, что на возвращаемость кредита влияет страхование ипотеки и обременение недвижимости банком. Кредитный рейтинг по этой группе можно повысить. 
- Наибольшее количество должников, взявших кредиты, с целями "на операции с автомобилем" и "на получение образования". Кредитный рейтинг по таким группам необходимо понизить.
	
**Для более достоверного и полноценного анализа платежеспособности клиентов при составлении математической модели кредитного скорринга необходимо применять повышающие и понижающие коэффициенты ко всем вышуказанным характеристикам, а также распределить вес каждого показателя для расчета итогового скорринг-балла.**

